In [ ]:
import numpy as np
import pandas as pd
import pickle

CONCISE = False

# Load the dataset
filename = "troop_movements_1m.csv"
data = pd.read_csv(filename)
df = pd.DataFrame(data)

# Replace 'invalid_unit' with unknown
df['unit_type'] = df['unit_type'].replace('invalid_unit', 'unknown')

# Fill missing location_x and location_y with ffill
df['location_x'] = df['location_x'].ffill()
df['location_y'] = df['location_y'].ffill()

# Save the clean data into Parquet format
df.to_parquet("troop_movements_1m_cleaned.parquet", index=False)

if CONCISE:
    display(df.head())
else:
    display(df)

In [ ]:
# Load pickled model
with open("trained_model.pkl", "rb") as file:
    model = pickle.load(file)
    
    # Load data from Parquet
    df = pd.read_parquet("troop_movements_1m_cleaned.parquet")
    
    X = df[['unit_type', 'homeworld']]
    X_enc = pd.get_dummies(X)
    
    # Run the data through the model and add the predicted values to the data frame
    df['predictions'] = model.predict(X_enc)

if CONCISE:
    display(df.head())
else:
    display(df)